Simple_RNN_scratch.

In [ ]:
def bin2int(bin_list):
  int_val = ""
  for k in bin_list:
    int_val += str(int(k))
  return int(int_val,2)

In [ ]:
def dataset(num):
  bin_len = 8
  X = np.zeros((num,bin_len))
  Y = np.zeros((num))
  for i in range (num):
    X[i] = np.round(np.random.rand(bin_len)).astype(int)
    Y[i] = bin2int(X[i])
  return X,Y

In [ ]:
no_of_samples = 20
trainX,trainY = dataset(no_of_samples)
testX,testY = dataset(5)

In [ ]:
class RNN:
  def __init__(self):
    self.w = [1,1]
    self.w_delta = [0.001, 0.001]
    self.w_sign = [0,0]
    self.eta_p = 1.2
    self.eta_n = 0.5
  def state (self, xk, sk):
    return xk * self.w[0] + sk * self.w[1]
  def forward_states(self, X):
    S = np.zeros((X.shape[0], X.shape[1] + 1))
    for k in range(0, X.shape[1]):
      next_state = self.state(X[:,k], S[:,k])
      S[:, k+1] = next_state
    return S
  def output_gradient(self,guess, real):
    return 2 * (guess - real) / no_of_samples
  def backward_gradient(self, X, S, grad_out):
    grad_over_time = np.zeros((X.shape[0], X.shape[1] + 1))
    grad_over_time[:,-1] = grad_out
    wx_grad = 0
    wr_grad = 0
    for k in range(X.shape[1],0 , -1):
      wx_grad += np.sum (grad_over_time[:,k] * X[:,k-1])
      wr_grad += np.sum (grad_over_time[:,k] * S[:,k-1])
      grad_over_time[:, k-1] = grad_over_time[:, k] * self.w[1]
    return (wx_grad, wr_grad), grad_over_time
  def update_rprop(self, X,Y, w_prev_sign, w_delta):
    S = self.forward_states(X)
    grad_out = self.output_gradient(S[:, -1] , Y)
    w_grads, _ = self.backward_gradient(X, S, grad_out)
    self.w_sign = np.sign(w_grads)
    for i, _ in enumerate(self.w):
      if self.w_sign[i] == w_prev_sign[i] :
        w_delta[i] *= self.eta_p
      else:
        w_delta[i] *= self.eta_n
    self.w_delta = w_delta
  def train(self, X, Y, training_epoches):
    for epoches in range(training_epoches):
      self.update_rprop(X, Y, self.w_sign, self.w_delta)
      for i, _ in enumerate(self.w):
        self.w[i]  -= self.w_sign[i] * self.w_delta[i]

In [ ]:
rnn = RNN()
rnn.train(trainX, trainY, 20000)
print("weight: \t", rnn.w)
print(" Real: \t", testY)

weight: 	 [1.0000000002263607, 1.9999999999274327]
 Real: 	 [ 48. 219. 138.  92. 243.]
